In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from pickletools import optimize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) =mnist.load_data()

In [2]:
x_train=tf.keras.utils.normalize(x_train,axis=1)
x_test=tf.keras.utils.normalize(x_test,axis=1)


In [3]:
x_train,x_eval,y_train,y_eval= train_test_split(x_train,y_train,test_size=0.2)

In [ ]:


def ANNmodel():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(optimizer= "Nadam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
    return model

In [ ]:
nums= range(128,480,64)
val_loss= np.inf
val_acc=0
num = 0
for i in range(1,6,2):
    # for val in nums:
        model = ANNmodel()
        model.fit(x_train,y_train,epochs=i,batch_size=32)
        loss,acc = model.evaluate(x_eval,y_eval)
        if loss <val_loss and acc >val_acc:
            model.save("best_ANNmodel1.model")


In [4]:
def build_model(hp):
   model = tf.keras.models.Sequential()

   model.add(tf.keras.layers.Conv2D(hp.Int('input_units',min_value=32, max_value=256,step =32), (3, 3), input_shape=(28,28,1)))
   model.add(tf.keras.layers.Activation('relu'))
   model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) 
   for i in range(hp.Int("n_layers",1,4)):
      model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_{i}_units',min_value=32, max_value=256,step =32), (3, 3)))
      model.add(tf.keras.layers.Activation('relu'))
   model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))   

   model.add(tf.keras.layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors 
   model.add(tf.keras.layers.Dense(10))
   model.add(tf.keras.layers.Activation("softmax"))    
   model.compile(optimizer="adam",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
   return model

In [7]:
from msilib.schema import Directory
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter
import time
# LOG_DIR = f"{int(time.time())}"
tuner= RandomSearch(build_model,objective='val_accuracy',max_trials=3)
tuner.search(x=x_train,y=y_train,epochs=50,batch_size=32,validation_data=(x_eval,y_eval),callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 2, restore_best_weights = True)],verbose = 1)


Trial 3 Complete [00h 01m 03s]
val_accuracy: 0.9862499833106995

Best val_accuracy So Far: 0.9899166822433472
Total elapsed time: 00h 02m 54s
INFO:tensorflow:Oracle triggered exit


In [8]:
print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())
print(tuner.get_best_models()[0])

{'input_units': 96, 'n_layers': 2, 'conv_0_units': 96, 'conv_1_units': 96}
Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
input_units: 96
n_layers: 2
conv_0_units: 96
conv_1_units: 96
Score: 0.9899166822433472
Trial summary
Hyperparameters:
input_units: 64
n_layers: 3
conv_0_units: 32
conv_1_units: 224
conv_2_units: 32
Score: 0.9862499833106995
Trial summary
Hyperparameters:
input_units: 64
n_layers: 2
conv_0_units: 224
conv_1_units: 32
Score: 0.9851666688919067
None


In [ ]:
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D

def CNNmodel():
   model =  tf.keras.models.Sequential()
   model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1)))
   model.add(Conv2D(64, (3, 3), activation='relu'))
   model.add(MaxPooling2D(pool_size=(2, 2)))
   model.add(Dropout(0.25))
   model.add(Flatten())
   model.add(Dense(256, activation='relu'))
   model.add(Dropout(0.5))
   model.add(Dense(10, activation='softmax'))
   model.compile(loss='sparse_categorical_crossentropy',optimizer='Nadam',metrics=['accuracy'])
   return model

In [ ]:
def Visualiztion(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


In [ ]:
Visualiztion(history)

In [ ]:
val_loss= np.inf
val_acc=0
num = 0
for i in range(3,11,2):
   # for val in nums:
   modelCNN = CNNmodel()
   x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
   x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
   modelCNN.fit(x_train,y_train,epochs=i,batch_size=64)
   loss,acc = modelCNN.evaluate(x_eval,y_eval)
   if loss <val_loss and acc >val_acc:
        modelCNN.save("best_CNNmodel1.model")
        val_loss=loss
        val_acc=val_acc

In [ ]:
new_model = tf.keras.models.load_model('best_ANNmodel1.model')
new_loss,new_acc = new_model.evaluate(x_test,y_test)
print(f"Best lost: {new_loss}, Best acc: {new_acc}, Best Number neurons: {512}")

In [ ]:
new_model = tf.keras.models.load_model('best_CNNmodel1.model')
new_loss,new_acc = new_model.evaluate(x_test,y_test)
print(f"Best lost: {new_loss}, Best acc: {new_acc}, Best Number neurons: {64}")

In [ ]:
new_model.predict

Part 2 

In [ ]:
import os 
import cv2
import matplotlib.pylab as plt
DATADIR=r'D:\sign to english\Youtube class\kagglecatsanddogs_5340\PetImages'
Categories = ['Dog','Cat']
for cate in Categories:
    path=os.path.join(DATADIR,cate)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        # plt.imshow(img_array,cmap='gray')
        # plt.show()

In [ ]:
IMG_SIZE= 50
new_array =cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))


In [ ]:
tf.keras.utils.plot_model(model, rankdir="LR", show_shapes=False, show_layer_names=False)